In [1]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

In [2]:
pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

In [3]:
%matplotlib notebook

In [4]:
df = pd.read_excel("赛题1数据材料/赛题1数据集/购地-市场交易-土地转让.xlsx", index_col="企业编号").drop_duplicates()
df

,成交时间,土地面积(公顷),土地用途,土地使用年限,土地级别,土地使用权类型,土地利用状况,转让方式,转让价格(万元)
企业编号,,,,,,,,,
1001,2010-07-14,0.018960,商务金融用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2010-07-14,0.012060,商务金融用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2010-07-14,0.049570,商务金融用地,NaN,NaN,NaN,NaN,NaN,NaN
1001,2012-11-20,0.762853,商务金融用地,NaN,NaN,NaN,"1、该宗地原于2009年1月划拨给长沙市滨江新城建设开发有限责任公司作为基础设施用地,其中",NaN,NaN
1001,2012-11-26,0.762853,商务金融用地,NaN,NaN,NaN,"根据《长沙市城乡规划局规划设计条件书》(案卷编号20120172XDGH1),用途为商业金",NaN,NaN
...,...,...,...,...,...,...,...,...,...
3995,2011-04-13,3.895493,工业用地,NaN,NaN,NaN,NaN,NaN,NaN
3998,2015-10-08,0.010798,商务金融用地,NaN,NaN,NaN,现状,NaN,NaN
3998,2007-09-20,0.670000,采矿用地,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def to_land_level(cell):
    if not isinstance(cell, str):
        return cell
    if '一级' in cell and '十一级' not in cell:
        return '一级'
    if '8' in cell:
        return '八级'
    if '1' in cell:
        return '一级'
    if '4' in cell:
        return '四级'
    if '2' in cell:
        return '二级'
    if '5' in cell:
        return '五级'
    return cell
    
    
land_level = pd.get_dummies(df['土地级别'].map(to_land_level), prefix='土地级别').groupby('企业编号').sum().T
land_level = land_level.reindex(columns=list(range(1001, 4001))).fillna(0)
land_level.index

Index(['土地级别_一级', '土地级别_七级', '土地级别_三级', '土地级别_九级', '土地级别_二级', '土地级别_五级',
       '土地级别_八级', '土地级别_六级', '土地级别_十一级', '土地级别_十三等级', '土地级别_十二级', '土地级别_十五等级',
       '土地级别_十八等级', '土地级别_十四等级', '土地级别_十级', '土地级别_四级', '土地级别_未评估地区'],
      dtype='object')

In [6]:
lst = df['土地使用权类型'].value_counts()[df['土地使用权类型'].value_counts() == 1].index


def to_land_use_right_type(cell):
    if not isinstance(cell, str):
        return cell
    if cell not in lst:
        return cell


land_use_right_type = pd.get_dummies(df['土地使用权类型'].map(to_land_use_right_type), prefix='土地使用权类型').groupby('企业编号').sum().T
land_use_right_type = land_use_right_type.reindex(columns=list(range(1001, 4001))).fillna(0)
land_use_right_type

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
土地使用权类型_出让,0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
土地使用权类型_出让(协议),0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
土地使用权类型_出让国有土地,0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
土地使用权类型_出让国有土地使用权,0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
土地使用权类型_出让国有建设用地使用权,0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
土地使用权类型_挂牌出让,0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
土地使用权类型_授权经营,0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
土地使用权类型_租赁,0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
土地使用权类型_转让,0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0


In [7]:
transfer_method = pd.get_dummies(df['转让方式'], prefix='转让方式').groupby('企业编号').sum().T
transfer_method = transfer_method.reindex(columns=list(range(1001, 4001))).fillna(0)
transfer_method.index

Index(['转让方式_1', '转让方式_3', '转让方式_6', '转让方式_7', '转让方式_买卖', '转让方式_交换',
       '转让方式_作价出资入股', '转让方式_其他', '转让方式_继承'],
      dtype='object')

In [8]:
pd.concat([land_level, land_use_right_type, transfer_method]).to_pickle('购地-市场交易-土地转让.pickle')